# Pacotes / Bibliotecas

In [3]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

NameError: name '_C' is not defined

# Ambiente

In [ ]:
enviroment_name = 'CartPole-v0'
env = gym.make(enviroment_name )

In [3]:
episodes = 5

for episode in range(1, episodes + 1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode: {}  Score: {}'.format(episode, score))
        
env.close()

Episode: 1  Score: 16.0
Episode: 2  Score: 27.0
Episode: 3  Score: 32.0
Episode: 4  Score: 15.0
Episode: 5  Score: 29.0


# Treinando o modelo AR

In [4]:
parent_dir = os.getcwd()
path = os.path.join(parent_dir,'Training')
try:
    os.mkdir(path)
    os.mkdir(os.path.join(path,'Logs'))
    os.mkdir(os.path.join(path,'Saved Models'))
except OSError as e:
    print(e)

[WinError 183] Não é possível criar um arquivo já existente: 'C:\\Users\\Pichau\\Jupyter Toys\\Training'


In [5]:
log_path = os.path.join('Training', 'Logs')

In [1]:
env = gym.make(enviroment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log = log_path)

NameError: name 'gym' is not defined

In [7]:
model.learn(total_timesteps = 20000)

Logging to Training\Logs\PPO_5
-----------------------------
| time/              |      |
|    fps             | 891  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 739         |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008386917 |
|    clip_fraction        | 0.0967      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00287    |
|    learning_rate        | 0.0003      |
|    loss                 | 9.23        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0167     |
|    value_loss           | 54.2        |
-----------------------------------------
---

# Salvando e recarregando o modelo

In [8]:
PPO_Path = os.path.join('Training', 'Saved Models', 'PPO_Model_Cartpole')
model.save(PPO_Path)

In [9]:
del model

In [10]:
model = PPO.load(PPO_Path)

# Avaliação

In [11]:
evaluate_policy(model, env, n_eval_episodes = 10, render = True)

C:\Users\Pichau\anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(200.0, 0.0)

In [12]:
env.close()

# Testando modelo

In [13]:
episodes = 5

for episode in range(1, episodes + 1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode: {}  Score: {}'.format(episode, score))
        
env.close()

Episode: 1  Score: [200.]
Episode: 2  Score: [200.]
Episode: 3  Score: [200.]
Episode: 4  Score: [200.]
Episode: 5  Score: [200.]


# Visualizando Logs com Tensorboard

In [18]:
training_path_log = os.path.join(log_path, 'PPO_2')

In [19]:
training_path_log

'Training\\Logs\\PPO_2'

In [23]:
!tensorboard --logdir={training_path_log}

^C


# Callback

In [14]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [15]:
save_path = os.path.join('Training', 'Saved Models')

In [17]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold = 200, verbose = 1)
eval_callback = EvalCallback(env,
                            callback_on_new_best = stop_callback,
                             eval_freq = 10000,
                             best_model_save_path = save_path,
                             verbose = 1)

In [18]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log = log_path)

Using cpu device


In [19]:
model.learn(total_timesteps = 20000, callback = eval_callback)

Logging to Training\Logs\PPO_6
-----------------------------
| time/              |      |
|    fps             | 1387 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 931         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008593759 |
|    clip_fraction        | 0.0847      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.687      |
|    explained_variance   | 0.00241     |
|    learning_rate        | 0.0003      |
|    loss                 | 5.35        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0136     |
|    value_loss           | 48.7        |
-----------------------------------------
---

# Mudando políticas

In [21]:
net_arch = [dict(pi = [128, 128, 128, 128], vf = [128, 128, 128, 128])]

In [22]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log = log_path, policy_kwargs = {'net_arch':net_arch})

Using cpu device


In [23]:
model.learn(total_timesteps = 20000, callback = eval_callback)

Logging to Training\Logs\PPO_7
-----------------------------
| time/              |      |
|    fps             | 923  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 611         |
|    iterations           | 2           |
|    time_elapsed         | 6           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.014406921 |
|    clip_fraction        | 0.216       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.681      |
|    explained_variance   | -0.00189    |
|    learning_rate        | 0.0003      |
|    loss                 | 3.45        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.027      |
|    value_loss           | 22          |
-----------------------------------------
---

# Mudando algoritmos

In [24]:
from stable_baselines3 import DQN

In [26]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log = log_path)

Using cpu device


In [27]:
model.learn(total_timesteps = 20000)

Logging to Training\Logs\DQN_1
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.957    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 5352     |
|    time_elapsed     | 0        |
|    total_timesteps  | 91       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.902    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 6443     |
|    time_elapsed     | 0        |
|    total_timesteps  | 206      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.853    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 6722     |
|    time_elapsed     | 0        |
|    total_timesteps  | 309      |
----------------------------------
------------------------

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 108      |
|    fps              | 4132     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2343     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 112      |
|    fps              | 3824     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2417     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 116      |
|    fps              | 3842     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2540     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 216      |
|    fps              | 4808     |
|    time_elapsed     | 0        |
|    total_timesteps  | 4751     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 220      |
|    fps              | 4817     |
|    time_elapsed     | 1        |
|    total_timesteps  | 4822     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 224      |
|    fps              | 4860     |
|    time_elapsed     | 1        |
|    total_timesteps  | 4933     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 324      |
|    fps              | 5263     |
|    time_elapsed     | 1        |
|    total_timesteps  | 7111     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 328      |
|    fps              | 5267     |
|    time_elapsed     | 1        |
|    total_timesteps  | 7200     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 332      |
|    fps              | 5277     |
|    time_elapsed     | 1        |
|    total_timesteps  | 7277     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 432      |
|    fps              | 4994     |
|    time_elapsed     | 1        |
|    total_timesteps  | 9449     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 436      |
|    fps              | 5002     |
|    time_elapsed     | 1        |
|    total_timesteps  | 9525     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 440      |
|    fps              | 5001     |
|    time_elapsed     | 1        |
|    total_timesteps  | 9587     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 540      |
|    fps              | 4964     |
|    time_elapsed     | 2        |
|    total_timesteps  | 11762    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 544      |
|    fps              | 4969     |
|    time_elapsed     | 2        |
|    total_timesteps  | 11858    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 548      |
|    fps              | 4968     |
|    time_elapsed     | 2        |
|    total_timesteps  | 11949    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 648      |
|    fps              | 5026     |
|    time_elapsed     | 2        |
|    total_timesteps  | 13969    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 652      |
|    fps              | 5036     |
|    time_elapsed     | 2        |
|    total_timesteps  | 14048    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 656      |
|    fps              | 5047     |
|    time_elapsed     | 2        |
|    total_timesteps  | 14159    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 756      |
|    fps              | 5186     |
|    time_elapsed     | 3        |
|    total_timesteps  | 16141    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 760      |
|    fps              | 5197     |
|    time_elapsed     | 3        |
|    total_timesteps  | 16231    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 764      |
|    fps              | 5203     |
|    time_elapsed     | 3        |
|    total_timesteps  | 16335    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 864      |
|    fps              | 5348     |
|    time_elapsed     | 3        |
|    total_timesteps  | 18512    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 868      |
|    fps              | 5357     |
|    time_elapsed     | 3        |
|    total_timesteps  | 18585    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 872      |
|    fps              | 5362     |
|    time_elapsed     | 3        |
|    total_timesteps  | 18677    |
----------------------------------
----------------------------------
| rollout/          